In [1]:
%load_ext autoreload
%autoreload 2

        
import ray
import ray.rllib.agents.ppo as ppo
from ray.tune.logger import pretty_print
from ray import tune
from ray.rllib.agents.ppo import PPOTrainer
from ray.rllib.models import FullyConnectedNetwork, Model, ModelCatalog

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import sys
sys.path.append("../../../")
sys.path.append("/home/ctripp/src/cavs-environments")
print(sys.path)

import pygame
import pymunk

import numpy as np
import math

 
import tensorflow as tf

import gym
import cavs_environments
import matplotlib

from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines.common.vec_env import SubprocVecEnv
from stable_baselines import PPO2
from stable_baselines import A2C
from stable_baselines import TRPO
from stable_baselines.common.policies import FeedForwardPolicy, register_policy
# from cavs_environments.vehicle.deep_road.deep_road import DeepRoad

import cavs_environments.framework as framework
import cavs_environments.vehicle.k_road.targeting as targeting
import cavs_environments.vehicle.k_road.road as road
# import  cavs_environments.vehicle.k_road.targeting as





ray.init()

import pprint
pp = pprint.PrettyPrinter(indent=4)

# ModelCatalog.register_custom_model("ThisRoadEnv", ThisRoadEnv)
# register_env("ThisRoadEnv", lambda config: ThisRoadEnv())

W0730 14:09:29.226424 139987772512064 deprecation.py:323] From /home/ctripp/miniconda3/envs/cavs-environments/lib/python3.7/site-packages/tensorflow/python/compat/v2_compat.py:61: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term


['/home/ctripp/miniconda3/envs/cavs-environments/lib/python3.7/site-packages/ray/pyarrow_files', '/home/ctripp/src/cavs-environments/cavs_environments/vehicle/k_road', '/home/ctripp/src/cavs-environments/cavs_environments/vehicle/k_road', '/home/ctripp/src/cavs-environments/cavs_environments/vehicle/k_road/~/project/flow/flow', '/home/ctripp/miniconda3/envs/cavs-environments/lib/python37.zip', '/home/ctripp/miniconda3/envs/cavs-environments/lib/python3.7', '/home/ctripp/miniconda3/envs/cavs-environments/lib/python3.7/lib-dynload', '', '/home/ctripp/.local/lib/python3.7/site-packages', '/home/ctripp/.local/lib/python3.7/site-packages/carla-0.9.5-py3.5-linux-x86_64.egg', '/home/ctripp/miniconda3/envs/cavs-environments/lib/python3.7/site-packages', '/home/ctripp/miniconda3/envs/cavs-environments/lib/python3.7/site-packages/carla-0.9.5-py3.7-linux-x86_64.egg', '/home/ctripp/miniconda3/envs/cavs-environments/lib/python3.7/site-packages/IPython/extensions', '/home/ctripp/.ipython', '/home/ct

2019-07-30 14:09:30,254	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-30_14-09-30_253880_20837/logs.
2019-07-30 14:09:30,367	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:31314 to respond...
2019-07-30 14:09:30,491	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:18472 to respond...
2019-07-30 14:09:30,498	INFO services.py:806 -- Starting Redis shard with 6.71 GB max memory.
2019-07-30 14:09:30,549	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-30_14-09-30_253880_20837/logs.
2019-07-30 14:09:30,552	INFO services.py:1446 -- Starting the Plasma object store with 10.07 GB memory using /dev/shm.


In [ ]:
import pygame
pygame.display.quit()


from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [2]:

# def make_target_env_with_baseline(
#     observation_scaling = 1.0, 
#     action_scaling = 1.0 / 10.0,
#     max_distance_from_target = 125, 
#     time_limit = 60):
    
#     return framework.FactoredGym(
#         targeting.TargetProcess(time_limit, max_distance_from_target),
#         targeting.TargetObserver(observation_scaling),
#         targeting.TargetTerminator(),
#         targeting.TargetRewarder(),
#         [framework.ActionScaler(action_scaling), targeting.TargetBaseline()]
#         )

class ThisRoadEnv(framework.FactoredGym):
    def __init__(self, env_config):
        observation_scaling = 1.0
        action_scaling = 1.0 / 10.0
        super().__init__(
        road.RoadProcess(),
        road.RoadObserver(observation_scaling),
        road.RoadTerminator(),
        road.RoadGoalRewarder(),
        [framework.ActionScaler(action_scaling)]
        )

def make_road_env(
    observation_scaling = 1.0,
    action_scaling = 1.0 / 10.0):
    return ThisRoadEnv(observation_scaling, action_scaling)

class CustomPolicy(FeedForwardPolicy):
    def __init__(self, *args, **kwargs):
        super(CustomPolicy, self).__init__(*args, **kwargs,
                                           net_arch=[64, 64, dict(pi=[64],
                                                          vf=[64])],
                                           feature_extraction="mlp")

# class CustomPolicy(MlpPolicy):
#     def __init__(self, *args, **kwargs):
#         super(MlpPolicy, self).__init__(*args, act_fun=tf.nn.tanh, net_arch=[32, 32])
        
# register_policy('LargeMLP', LargeMLP)


In [3]:
tune.run(
    "PPO",
    stop={
        "timesteps_total": 10000
    },
    config={
        "env": ThisRoadEnv,
        "model":{
            "fcnet_hiddens":[64,64]
        },
        "num_workers": 1
    })


2019-07-30 14:09:40,516	INFO tune.py:61 -- Tip: to resume incomplete experiments, pass resume='prompt' or resume=True to run()
2019-07-30 14:09:40,517	INFO tune.py:233 -- Starting a new experiment.
2019-07-30 14:09:40,532	WARNING signature.py:108 -- The function with_updates has a **kwargs argument, which is currently not supported.
W0730 14:09:40.540739 139987772512064 deprecation_wrapper.py:119] From /home/ctripp/miniconda3/envs/cavs-environments/lib/python3.7/site-packages/ray/tune/logger.py:136: The name tf.VERSION is deprecated. Please use tf.version.VERSION instead.

W0730 14:09:40.541399 139987772512064 deprecation_wrapper.py:119] From /home/ctripp/miniconda3/envs/cavs-environments/lib/python3.7/site-packages/ray/tune/logger.py:141: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/12 CPUs, 0/1 GPUs
Memory usage on this node: 6.9/33.6 GB

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 2/12 CPUs, 0/1 GPUs
Memory usage on this node: 6.9/33.6 GB
Result logdir: /home/ctripp/ray_results/PPO
Number of trials: 1 ({'RUNNING': 1})
RUNNING trials:
 - PPO_ThisRoadEnv_0:	RUNNING



2019-07-30 14:09:41,602	ERROR trial_runner.py:487 -- Error processing event.
Traceback (most recent call last):
  File "/home/ctripp/miniconda3/envs/cavs-environments/lib/python3.7/site-packages/ray/tune/trial_runner.py", line 436, in _process_trial
    result = self.trial_executor.fetch_result(trial)
  File "/home/ctripp/miniconda3/envs/cavs-environments/lib/python3.7/site-packages/ray/tune/ray_trial_executor.py", line 323, in fetch_result
    result = ray.get(trial_future[0])
  File "/home/ctripp/miniconda3/envs/cavs-environments/lib/python3.7/site-packages/ray/worker.py", line 2195, in get
    raise value
ray.exceptions.RayTaskError: ray_PPO:train() (pid=20926, host=nrel-34256s)
  File "pyarrow/serialization.pxi", line 461, in pyarrow.lib.deserialize
  File "pyarrow/serialization.pxi", line 424, in pyarrow.lib.deserialize_from
  File "pyarrow/serialization.pxi", line 275, in pyarrow.lib.SerializedPyObject.deserialize
  File "pyarrow/serialization.pxi", line 174, in pyarrow.lib.Seria

(pid=20926) WARNING: Logging before flag parsing goes to stderr.
(pid=20926) W0730 14:09:41.435480 139781480658752 deprecation.py:323] From /home/ctripp/miniconda3/envs/cavs-environments/lib/python3.7/site-packages/tensorflow/python/compat/v2_compat.py:61: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=20926) Instructions for updating:
(pid=20926) non-resource variables are not supported in the long term
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/12 CPUs, 0/1 GPUs
Memory usage on this node: 7.1/33.6 GB
Result logdir: /home/ctripp/ray_results/PPO
Number of trials: 1 ({'ERROR': 1})
ERROR trials:
 - PPO_ThisRoadEnv_0:	ERROR, 1 failures: /home/ctripp/ray_results/PPO/PPO_ThisRoadEnv_0_2019-07-30_14-09-40t3jmnxkf/error_2019-07-30_14-09-41.txt



TuneError: ('Trials did not complete', [PPO_ThisRoadEnv_0])

In [4]:


# env = make_road_env()

# config = ppo.DEFAULT_CONFIG.copy()
# config.
config = config={
    "env_config": {},  # config to pass to env class
}
config["num_gpus"] = 1
config["num_workers"] = 1
# config['use_eager'] = True
# config['stop'] = 10e3
# config['model'] = {"custom_model":"my_model"}
trainer = ppo.PPOTrainer(config=config, env=ThisRoadEnv)

for i in range(1000):
   # Perform one iteration of training the policy with PPO
   result = trainer.train()
   print(pretty_print(result))

   if i % 100 == 0:
       checkpoint = trainer.save()
       print("checkpoint saved at", checkpoint)


2019-07-30 14:12:56,818	WARNING ppo.py:151 -- FYI: By default, the value function will not share layers with the policy model ('vf_share_layers': False).
2019-07-30 14:12:56,853	INFO rollout_worker.py:301 -- Creating policy evaluation worker 0 on CPU (please ignore any CUDA init errors)
W0730 14:12:56.884514 139987772512064 deprecation.py:323] From /home/ctripp/miniconda3/envs/cavs-environments/lib/python3.7/site-packages/ray/rllib/models/fcnet.py:37: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.
W0730 14:12:57.081722 139987772512064 deprecation.py:323] From /home/ctripp/miniconda3/envs/cavs-environments/lib/python3.7/site-packages/ray/rllib/models/action_dist.py:172: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
2019-07-30 14:12:57,150	INFO dynamic_tf_policy.py:313 -- Ini

(pid=20919) WARNING: Logging before flag parsing goes to stderr.
(pid=20919) W0730 14:12:58.772580 140541510301504 deprecation.py:323] From /home/ctripp/miniconda3/envs/cavs-environments/lib/python3.7/site-packages/tensorflow/python/compat/v2_compat.py:61: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=20919) Instructions for updating:
(pid=20919) non-resource variables are not supported in the long term


W0730 14:12:59.636049 139987772512064 deprecation.py:323] From /home/ctripp/miniconda3/envs/cavs-environments/lib/python3.7/site-packages/ray/rllib/policy/tf_policy.py:542: Variable.load (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.
2019-07-30 14:12:59,669	INFO ppo.py:106 -- Important! Since 0.7.0, observation normalization is no longer enabled by default. To enable running-mean normalization, set 'observation_filter': 'MeanStdFilter'. You can ignore this message if your environment doesn't require observation normalization.
2019-07-30 14:13:00,046	INFO trainer.py:361 -- Worker crashed during call to train(). To attempt to continue training without the failed worker, set `'ignore_worker_failures': True`.


RayTaskError: [36mray_RolloutWorker:sample()[39m (pid=20919, host=nrel-34256s)
  File "pyarrow/serialization.pxi", line 461, in pyarrow.lib.deserialize
  File "pyarrow/serialization.pxi", line 424, in pyarrow.lib.deserialize_from
  File "pyarrow/serialization.pxi", line 275, in pyarrow.lib.SerializedPyObject.deserialize
  File "pyarrow/serialization.pxi", line 174, in pyarrow.lib.SerializationContext._deserialize_callback
ModuleNotFoundError: No module named 'cavs_environments'


In [ ]:
ego_starting_distance = 10.0
target_reward = 90
iter = 0
step = 1e3

inner_env = make_road_env()
inner_env.process.ego_starting_distance = ego_starting_distance
inner_env.reset()

env = DummyVecEnv([lambda: inner_env])  # The algorithms require a vectorized environment to run
model = PPO2(CustomPolicy, env, verbose=0, tensorboard_log='/tmp/k_road_0/',
            gamma=.999, learning_rate=.001)

while iter < 250e3:
    
    inner_env.process.ego_starting_distance  = ego_starting_distance
    model.learn(total_timesteps=int(step), reset_num_timesteps = False)
    
    print('er: ', model.episode_reward)
    mean_reward = np.mean(model.episode_reward)
    
    if mean_reward >= target_reward:
        ego_starting_distance = min(600.0, ego_starting_distance + 10.0)
        
    print('iter: ', iter, 'reward: ', mean_reward, 'starting_distance: ', inner_env.process.ego_starting_distance)
    
    iter = iter + step
model.save('k_road_0')
print('done!')

In [ ]:
env = make_road_env()
env.process.ego_starting_distance = 10

env = DummyVecEnv([lambda: env])  # The algorithms require a vectorized environment to run
model = PPO2.load('k_road_0')
for i in range(50):
    obs = env.reset()
    for i in range(100000):
        action, _states = model.predict(obs)
        obs, rewards, terminal, info = env.step(action)
        env.render()
        if terminal:
            break
env.close()

In [ ]:
env = make_road_env()
env.ego_starting_distance = 10
print(env.process.ego_vehicle, env.process.road_length, env.process.ego_starting_distance)

obs = None
for _ in range(20):
    env.reset()
    env.render()
    for _ in range(2000):
        action = np.empty(2)
        action[0] = .5 # np.random.normal(.5, .001)
        action[1] = 0 # np.random.normal(0, .01)

        result = env.step(action)
        obs = result[0]
        env.render()
        if result[2]:
            break
       
env.close()

In [ ]:
n_cpu = 10
env = SubprocVecEnv([lambda: make_road_env() for i in range(n_cpu)])
model = PPO2(MlpPolicy, env, verbose=1, tensorboard_log='/tmp/k_road_0/',
            gamma=.99999, learning_rate=.001)
model.learn(total_timesteps=int(200e3))
model.save('k_road_0')
print('done!')

In [ ]:
env = make_road_env()
env = DummyVecEnv([lambda: env])  # The algorithms require a vectorized environment to run
model = PPO2.load('k_road_0')
obs = env.reset()
for i in range(100000):
    action, _states = model.predict(obs)
    obs, rewards, terminal, info = env.step(action)
    env.render()
    if terminal:
        break
env.close()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as lines

def two_sided_offset_exponential(gain, offset, x):
    y = 0
    s = 1
    if x < offset:
        y = 1 - (x + 1) / (offset + 1)
        s = -1
    else:
        y = (x - offset) / (1 - offset)
    return s * (math.exp(gain * y) - 1) / (math.exp(gain) - 1)

fig = plt.figure()

x = np.arange(-1.0, 1.0, .01)
y = np.array([two_sided_offset_exponential(.1, -.5, xi) for xi in x])
plt.plot(x,y)
plt.show()


In [ ]:
import ray
from ray import tune
from ray.rllib.agents.ppo import PPOTrainer

def train(config, reporter):
    trainer = PPOTrainer(config=config, env=YourEnv)
    while True:
        result = trainer.train()
        reporter(**result)
        if result["episode_reward_mean"] > 200:
            phase = 2
        elif result["episode_reward_mean"] > 100:
            phase = 1
        else:
            phase = 0
        trainer.workers.foreach_worker(
            lambda ev: ev.foreach_env(
                lambda env: env.set_phase(phase)))

ray.init()
tune.run(
    train,
    config={
        "num_gpus": 0,
        "num_workers": 2,
    },
    resources_per_trial={
        "cpu": 1,
        "gpu": lambda spec: spec.config.num_gpus,
        "extra_cpu": lambda spec: spec.config.num_workers,
    },
)